In [679]:
# import pandas as pd

# df = pd.read_csv('ES_Quotes.csv')

In [680]:
# days = [str(i) for i in range(1, 21)]
# for i, day in enumerate(days):
#     if len(day) == 1:
#         days[i] = "0"+day

# es_days = [df[df['Date'].str.contains(f'09/{day}/2013')==True] for day in days]

In [681]:
# for i, df in enumerate(es_days):
#     df.to_csv(f'ES_ALL_QUOTES/es_{i+1}.csv')

In [682]:
j = 20

In [683]:
day_quotes = pd.read_csv(f'ES_ALL_QUOTES/es_{j}.csv')


In [684]:
day_quotes['Time'] = day_quotes['Time'].apply(lambda x: x + "000")


In [685]:
day_quotes.Time = pd.to_datetime(day_quotes.Time, format="%H:%M:%S.%f")

In [686]:
day_quotes.Time

0         1900-01-01 00:00:01.341
1         1900-01-01 00:00:01.341
2         1900-01-01 00:00:01.341
3         1900-01-01 00:00:01.341
4         1900-01-01 00:00:04.236
                    ...          
1616611   1900-01-01 16:14:59.111
1616612   1900-01-01 16:14:59.251
1616613   1900-01-01 16:14:59.753
1616614   1900-01-01 16:14:59.801
1616615   1900-01-01 16:14:59.875
Name: Time, Length: 1616616, dtype: datetime64[ns]

In [687]:
column_names = [f"d{i+1}" for i in range(20)]
column_names.append("last")


In [688]:
condensed_1s = []
current_start_time = day_quotes.Time[0]
start_px = (float(day_quotes['Ask Price'][0]) + float(day_quotes['Bid Price'][0])) / 2
for i, row in day_quotes.iterrows():
    if row.Time > current_start_time + pd.Timedelta(10000, 'millis'):
        mid_px = (float(day_quotes['Ask Price'][i-1]) + float(day_quotes['Bid Price'][i-1])) / 2
        px_delta = mid_px - start_px
        condensed_1s.append(px_delta)
        current_start_time = current_start_time + pd.Timedelta(10000, 'millis')
        start_px = mid_px


In [689]:
es_close_sequences = pd.DataFrame(columns=column_names, dtype=object)

In [690]:
i = 0
while i + 21 < len(condensed_1s): 
    es_close_sequences = es_close_sequences.append(pd.Series(condensed_1s[i:i+21], index=column_names), ignore_index=True)
    i+=21

In [691]:
len(condensed_1s)

5849

In [692]:
es_close_sequences

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,...,d12,d13,d14,d15,d16,d17,d18,d19,d20,last
0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,0.000,0.000,0.00,0.0,0.00,0.00,0.00,0.0,0.000
1,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,0.000,0.000,0.00,0.0,0.00,0.00,0.00,0.0,0.000
2,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,0.000,0.000,0.00,0.0,0.00,0.00,0.00,0.0,0.000
3,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,0.000,0.000,0.00,0.0,0.00,0.00,0.00,0.0,0.000
4,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,0.000,0.000,0.00,0.0,0.00,0.00,0.00,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.25,0.0,0.000,...,0.00,0.000,0.000,0.00,0.0,0.00,0.25,0.00,0.0,0.000
274,-0.25,0.0,0.25,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,-0.125,0.125,0.00,0.0,0.25,0.00,-0.25,0.0,0.250
275,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000,...,0.00,0.000,-0.250,0.00,0.0,0.00,0.00,0.00,0.0,0.000
276,0.00,0.0,-0.25,0.25,0.0,0.0,0.0,0.25,0.0,0.000,...,0.00,0.250,-0.250,0.00,0.0,0.00,0.00,0.00,0.0,0.000


In [693]:
es_close_sequences.to_csv(f'cleaned_data/es{j}_disjoint_10s.csv')